# Import data from yfinance

In [1]:
import yfinance as yf

In [33]:
# Get the data for the stock portfolio
portfolio = yf.Tickers('AMD')

# Get the historical data for the stock portfolio
hist = portfolio.history(start='2020-01-01', period='5y')


[*********************100%***********************]  1 of 1 completed


In [34]:
hist_amd = hist.xs('AMD', axis=1, level=1)
hist_amd

Price,Close,Dividends,High,Low,Open,Stock Splits,Volume
Date,,,,,,,
2020-01-02,49.099998,0.0,49.250000,46.630001,46.860001,0.0,80331100
2020-01-03,48.599998,0.0,49.389999,47.540001,48.029999,0.0,73127400
2020-01-06,48.389999,0.0,48.860001,47.860001,48.020000,0.0,47934900
2020-01-07,48.250000,0.0,49.389999,48.040001,49.349998,0.0,58061400
2020-01-08,47.830002,0.0,48.299999,47.139999,47.849998,0.0,53767000
...,...,...,...,...,...,...,...
2024-12-24,126.290001,0.0,127.510002,124.660004,127.510002,0.0,24829600
2024-12-26,125.059998,0.0,127.300003,125.050003,125.519997,0.0,24998200
2024-12-27,125.190002,0.0,126.180000,122.260002,124.370003,0.0,32924000


backtesting

In [37]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(hist_amd, SmaCross, commission=.002,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

/var/folders/zk/99phrd0d2fs5xjbc622wn4_h0000gn/T/ipykernel_41700/1382971500.py:22: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()


GridPlot(id='p4217', ...)

## Date-based buy/sell strategy
Given lists of buy and sell dates (we buy/sell the whole portfolio on those dates),
this Strategy maps the provided calendar dates to the nearest bars present in the OHLC data and
issues full-portfolio buys and full closes on matching bars.

Notes:
- Provide `buy_dates` and `sell_dates` as lists of date-like strings or pd.Timestamp objects when calling `Backtest.run(...)`.
- Dates that don't fall on trading days are matched by calendar date against available data index entries (i.e. the strategy finds bars whose normalized date equals the provided date).

In [46]:
from backtesting import Strategy
import pandas as pd

class DateStrategy(Strategy):
    # declare parameters so Backtest.run can override them
    buy_dates = None
    sell_dates = None

    def init(self):
        # Normalize incoming date lists to pandas.Timestamp (date-only) for robust matching
        # Accept None, list of strings, or list of Timestamp-like objects
        self._buy_dates = set(pd.to_datetime(self.buy_dates).normalize()) if self.buy_dates is not None else set()
        self._sell_dates = set(pd.to_datetime(self.sell_dates).normalize()) if self.sell_dates is not None else set()

        # Map provided dates to actual timestamps present in the OHLC index so we only trigger on available bars
        idx = pd.to_datetime(self.data.index)
        self._buy_bars = set(ts for ts in idx if ts.normalize() in self._buy_dates)
        self._sell_bars = set(ts for ts in idx if ts.normalize() in self._sell_dates)

    def next(self):
        # Current (most recent) bar timestamp available inside next()
        now = self.data.index[-1]

        # BUY whole portfolio if today is a buy-bar and we are flat
        if now in self._buy_bars:
            if not self.position:
                # buy full equity (default size behavior)
                self.buy()

        # SELL / close whole portfolio if today is a sell-bar and we have a position
        elif now in self._sell_bars:
            if self.position:
                # close entire position
                self.position.close()

# End of DateStrategy

In [47]:
# Example: run the date-driven strategy using the `hist_amd` dataframe already present in this notebook
# Replace the example buy/sell dates below with your actual lists.
from backtesting import Backtest

# Example buy/sell dates (calendar dates). Use YYYY-MM-DD strings or Timestamps.
buy_dates = ['2020-02-03', '2020-08-10', '2021-01-04', '2022-05-12']
sell_dates = ['2020-05-01', '2020-12-15', '2021-03-01']

# Create Backtest. Use exclusive_orders=True to ensure single position at a time.
bt = Backtest(hist_amd, DateStrategy, cash=10_000, exclusive_orders=True, finalize_trades=True)
stats = bt.run(buy_dates=buy_dates, sell_dates=sell_dates)

# print(stats[['Start', 'End', '# Trades', 'Equity Final [$]']])

# Optional: show interactive plot (will open a browser window)
bt.plot()

GridPlot(id='p5647', ...)